In [1]:
%matplotlib inline

# Preliminaries 

In [2]:
import os
import pandas as pd
from ase.io import read
import torch
import time
import numpy as np
from sklearn import preprocessing, linear_model, pipeline, model_selection

In [3]:
from sklearn import (linear_model, model_selection, preprocessing,
                     pipeline)
from scipy.spatial.distance import pdist

In [4]:
from kymatio.torch import HarmonicScattering3D

In [5]:
from kymatio.scattering3d.backend.torch_backend \
    import TorchBackend3D

In [6]:
from kymatio.scattering3d.utils \
    import generate_weighted_sum_of_gaussians

In [7]:
from kymatio.datasets import fetch_qm7
from kymatio.caching import get_cache_dir

# Energies dataframe

In [8]:
# Path setup
data_dir = "./"  # <-- Change this to your actual data path
atoms_train_dir = os.path.join(data_dir, "atoms", "train")
energies_csv = os.path.join(data_dir, "energies", "train.csv")

In [9]:
# Read energies CSV
energies_df = pd.read_csv(energies_csv)
energies_df["id"] = energies_df["id"].astype(str)
energies_df = energies_df.sort_values("id")
print("Loaded energy data:\n", energies_df.head().to_string(index=False))

Loaded energy data:
   id     energy
   1 -90.107880
  10 -69.210846
 100 -64.983899
1000 -76.559740
1001 -62.270961


# Positions and charges 

In [11]:
def extract_features(xyz_path):
    atoms = read(xyz_path)
    positions = atoms.get_positions()
    charges = atoms.get_atomic_numbers()
    return positions, charges

def load_all_xyz(folder_path, max_atoms=23):
    positions_list = []
    charges_list = []
    
    xyz_files = sorted([
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if f.endswith('.xyz')
    ])

    for xyz_path in xyz_files:
        pos, chg = extract_features(xyz_path)
        
        pos_padded = np.zeros((max_atoms, 3))
        chg_padded = np.zeros((max_atoms,))
        
        n_atoms = pos.shape[0]
        pos_padded[:n_atoms, :] = pos
        chg_padded[:n_atoms] = chg

        positions_list.append(pos_padded)
        charges_list.append(chg_padded)

    positions_array = np.stack(positions_list)  # shape (N, max_atoms, 3)
    charges_array = np.stack(charges_list)      # shape (N, max_atoms)
    
    return positions_array, charges_array

In [11]:
pos_train, full_charges_train = load_all_xyz("./atoms/train/", max_atoms=23)
n_molecules_train = pos_train.shape[0]
print(n_molecules_train)

6591


# Data preparation

In [12]:
mask = full_charges_train <= 2
valence_charges = full_charges_train* mask

mask = np.logical_and(full_charges_train > 2, full_charges_train <= 10)
valence_charges += (full_charges_train - 2) * mask

mask = np.logical_and(full_charges_train > 10, full_charges_train <= 18)
valence_charges += (full_charges_train - 10) * mask

In [13]:
overlapping_precision = 1e-1
sigma = 2.0
min_dist = np.inf

for i in range(n_molecules_train):
    n_atoms = np.sum(full_charges_train[i] != 0)
    pos_i = pos_train[i, :n_atoms, :]
    min_dist = min(min_dist, pdist(pos_i).min())

delta = sigma * np.sqrt(-8 * np.log(overlapping_precision))
pos = pos_train * delta / min_dist

## Scattering Transform



In [14]:
M, N, O = 96, 64, 48

grid = np.mgrid[-M//2:-M//2+M, -N//2:-N//2+N, -O//2:-O//2+O]
grid = np.fft.ifftshift(grid)

In [15]:
J = 2
L = 3
integral_powers = [0.5, 1.0, 2.0, 3.0]

scattering_train = HarmonicScattering3D(J=J, shape=(M, N, O),
                                  L=L, sigma_0=sigma,
                                  integral_powers=integral_powers)

In [16]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
scattering_train.to(device)
print("Using", device)

Using cuda


The maps computed for each molecule are quite large, so the computation has
to be done by batches. Here we select a small batch size to ensure that we
have enough memory when running on the GPU. Dividing the number of molecules
by the batch size then gives us the number of batches.



In [17]:
batch_size = 8
n_batches = int(np.ceil(n_molecules_train/ batch_size))
print(n_batches)

824


We are now ready to compute the scattering transforms. In the following
loop, each batch of molecules is transformed into three maps using Gaussians
centered at the atomic positions, one for the nuclear charges, one for the
valence charges, and one with their difference (called the “core” charges).
For each map, we compute its scattering transform up to order two and store
the results.



In [18]:
order_0, orders_1_and_2 = [], []
print('Computing solid harmonic scattering coefficients of '
      '{} molecules from the QM7 database on {}'.format(
        n_molecules_train,   "GPU" if use_cuda else "CPU"))
print('sigma: {}, L: {}, J: {}, integral powers: {}'.format(
        sigma, L, J, integral_powers))

this_time = None
last_time = None
for i in range(n_batches):
    this_time = time.time()
    if last_time is not None:
        dt = this_time - last_time
        print("Iteration {} ETA: [{:02}:{:02}:{:02}]".format(
                    i + 1, int(((n_batches - i - 1) * dt) // 3600),
                    int((((n_batches - i - 1) * dt) // 60) % 60),
                    int(((n_batches - i - 1) * dt) % 60)))
    else:
        print("Iteration {} ETA: {}".format(i + 1, '-'))
    last_time = this_time
    time.sleep(1)

    # Extract the current batch.
    start = i * batch_size
    end = min(start + batch_size, n_molecules_train)

    pos_batch = pos[start:end]
    full_batch = full_charges_train[start:end]
    val_batch = valence_charges[start:end]

    # Calculate the density map for the nuclear charges and transfer
    # to PyTorch.
    full_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, full_batch, sigma)
    full_density_batch = torch.from_numpy(full_density_batch)
    full_density_batch = full_density_batch.to(device).float()

    # Compute zeroth-order, first-order, and second-order scattering
    # coefficients of the nuclear charges.
    full_order_0 = TorchBackend3D.compute_integrals(full_density_batch,
                                     integral_powers)
    full_scattering = scattering_train(full_density_batch)

    # Compute the map for valence charges.
    val_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, val_batch, sigma)
    val_density_batch = torch.from_numpy(val_density_batch)
    val_density_batch = val_density_batch.to(device).float()

    # Compute scattering coefficients for the valence charges.
    val_order_0 = TorchBackend3D.compute_integrals(val_density_batch,
                                    integral_powers)
    val_scattering = scattering_train(val_density_batch)

    # Take the difference between nuclear and valence charges, then
    # compute the corresponding scattering coefficients.
    core_density_batch = full_density_batch - val_density_batch

    core_order_0 = TorchBackend3D.compute_integrals(core_density_batch,
                                     integral_powers)
    core_scattering = scattering_train(core_density_batch)

    # Stack the nuclear, valence, and core coefficients into arrays
    # and append them to the output.
    batch_order_0 = torch.stack(
        (full_order_0, val_order_0, core_order_0), dim=-1)
    batch_orders_1_and_2 = torch.stack(
        (full_scattering, val_scattering, core_scattering), dim=-1)

    order_0.append(batch_order_0)
    orders_1_and_2.append(batch_orders_1_and_2)

Computing solid harmonic scattering coefficients of 6591 molecules from the QM7 database on GPU
sigma: 2.0, L: 3, J: 2, integral powers: [0.5, 1.0, 2.0, 3.0]
Iteration 1 ETA: -
Iteration 2 ETA: [00:51:26]
Iteration 3 ETA: [00:45:04]
Iteration 4 ETA: [00:44:36]
Iteration 5 ETA: [00:45:47]
Iteration 6 ETA: [00:45:38]
Iteration 7 ETA: [00:43:29]
Iteration 8 ETA: [00:46:10]
Iteration 9 ETA: [00:46:15]
Iteration 10 ETA: [00:44:02]
Iteration 11 ETA: [00:43:33]
Iteration 12 ETA: [00:47:02]
Iteration 13 ETA: [00:44:57]
Iteration 14 ETA: [00:42:57]
Iteration 15 ETA: [00:45:49]
Iteration 16 ETA: [00:42:50]
Iteration 17 ETA: [00:43:24]
Iteration 18 ETA: [00:43:07]
Iteration 19 ETA: [00:44:24]
Iteration 20 ETA: [00:42:37]
Iteration 21 ETA: [00:42:28]
Iteration 22 ETA: [00:45:00]
Iteration 23 ETA: [00:43:38]
Iteration 24 ETA: [00:42:29]
Iteration 25 ETA: [00:44:38]
Iteration 26 ETA: [00:46:30]
Iteration 27 ETA: [00:44:01]
Iteration 28 ETA: [00:43:26]
Iteration 29 ETA: [00:43:08]
Iteration 30 ETA: [

In [19]:
order_0_train = torch.cat(order_0, dim=0)
orders_1_and_2_train = torch.cat(orders_1_and_2, dim=0)

order_0_train = order_0_train.cpu().numpy()
orders_1_and_2_train = orders_1_and_2_train.cpu().numpy()

In [20]:
order_0_train = order_0_train.reshape((n_molecules_train, -1))
orders_1_and_2_train = orders_1_and_2_train.reshape((n_molecules_train, -1))

In [21]:
basename = 'molecule_L_{}_J_{}_sigma_{}_MNO_{}_powers_{}.npy'.format(
        L, J, sigma, (M, N, O), integral_powers)

cache_dir = get_cache_dir("results/train/")

filename = os.path.join(cache_dir, 'order_0_' + basename)
np.save(filename, order_0_train)

filename = os.path.join(cache_dir, 'orders_1_and_2' + basename)
np.save(filename, orders_1_and_2_train)

# Test

In [12]:
pos_test, full_charges_test = load_all_xyz("./atoms/test/", max_atoms=23)
n_molecules_test = pos_test.shape[0]

In [13]:
mask = full_charges_test <= 2
valence_charges = full_charges_test * mask

mask = np.logical_and(full_charges_test > 2, full_charges_test <= 10)
valence_charges += (full_charges_test - 2) * mask

mask = np.logical_and(full_charges_test > 10, full_charges_test <= 18)
valence_charges += (full_charges_test - 10) * mask

In [14]:
overlapping_precision = 1e-1
sigma = 2.0
min_dist = np.inf

for i in range(n_molecules_test):
    n_atoms = np.sum(full_charges_test[i] != 0)
    pos_i = pos_test[i, :n_atoms, :]
    min_dist = min(min_dist, pdist(pos_i).min())

delta = sigma * np.sqrt(-8 * np.log(overlapping_precision))
pos = pos_test * delta / min_dist

In [15]:
M, N, O = 96, 64, 48

grid = np.mgrid[-M//2:-M//2+M, -N//2:-N//2+N, -O//2:-O//2+O]
grid = np.fft.ifftshift(grid)

In [16]:
J = 2
L = 3
integral_powers = [0.5, 1.0, 2.0, 3.0]

scattering = HarmonicScattering3D(J=J, shape=(M, N, O),
                                  L=L, sigma_0=sigma,
                                  integral_powers=integral_powers)

In [17]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
scattering.to(device)

HarmonicScattering3D()

In [18]:
batch_size = 8
n_batches = int(np.ceil(n_molecules_test / batch_size))
print(n_batches)

206


In [19]:
order_0, orders_1_and_2 = [], []
print('Computing solid harmonic scattering coefficients of '
      '{} molecules from the QM7 database on {}'.format(
        n_molecules_test,   "GPU" if use_cuda else "CPU"))
print('sigma: {}, L: {}, J: {}, integral powers: {}'.format(
        sigma, L, J, integral_powers))

this_time = None
last_time = None
for i in range(n_batches):
    this_time = time.time()
    if last_time is not None:
        dt = this_time - last_time
        print("Iteration {} ETA: [{:02}:{:02}:{:02}]".format(
                    i + 1, int(((n_batches - i - 1) * dt) // 3600),
                    int((((n_batches - i - 1) * dt) // 60) % 60),
                    int(((n_batches - i - 1) * dt) % 60)))
    else:
        print("Iteration {} ETA: {}".format(i + 1, '-'))
    last_time = this_time
    time.sleep(1)

    # Extract the current batch.
    start = i * batch_size
    end = min(start + batch_size, n_molecules_test)

    pos_batch = pos[start:end]
    full_batch = full_charges_test[start:end]
    val_batch = valence_charges[start:end]

    # Calculate the density map for the nuclear charges and transfer
    # to PyTorch.
    full_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, full_batch, sigma)
    full_density_batch = torch.from_numpy(full_density_batch)
    full_density_batch = full_density_batch.to(device).float()

    # Compute zeroth-order, first-order, and second-order scattering
    # coefficients of the nuclear charges.
    full_order_0 = TorchBackend3D.compute_integrals(full_density_batch,
                                     integral_powers)
    full_scattering = scattering(full_density_batch)

    # Compute the map for valence charges.
    val_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, val_batch, sigma)
    val_density_batch = torch.from_numpy(val_density_batch)
    val_density_batch = val_density_batch.to(device).float()

    # Compute scattering coefficients for the valence charges.
    val_order_0 = TorchBackend3D.compute_integrals(val_density_batch,
                                    integral_powers)
    val_scattering = scattering(val_density_batch)

    # Take the difference between nuclear and valence charges, then
    # compute the corresponding scattering coefficients.
    core_density_batch = full_density_batch - val_density_batch

    core_order_0 = TorchBackend3D.compute_integrals(core_density_batch,
                                     integral_powers)
    core_scattering = scattering(core_density_batch)

    # Stack the nuclear, valence, and core coefficients into arrays
    # and append them to the output.
    batch_order_0 = torch.stack(
        (full_order_0, val_order_0, core_order_0), dim=-1)
    batch_orders_1_and_2 = torch.stack(
        (full_scattering, val_scattering, core_scattering), dim=-1)

    order_0.append(batch_order_0)
    orders_1_and_2.append(batch_orders_1_and_2)

Computing solid harmonic scattering coefficients of 1647 molecules from the QM7 database on GPU
sigma: 2.0, L: 3, J: 2, integral powers: [0.5, 1.0, 2.0, 3.0]
Iteration 1 ETA: -
Iteration 2 ETA: [00:10:51]
Iteration 3 ETA: [00:09:22]
Iteration 4 ETA: [00:09:35]
Iteration 5 ETA: [00:09:15]
Iteration 6 ETA: [00:09:16]
Iteration 7 ETA: [00:09:08]
Iteration 8 ETA: [00:09:06]
Iteration 9 ETA: [00:09:03]
Iteration 10 ETA: [00:09:06]
Iteration 11 ETA: [00:08:59]
Iteration 12 ETA: [00:08:47]
Iteration 13 ETA: [00:08:47]
Iteration 14 ETA: [00:08:44]
Iteration 15 ETA: [00:08:41]
Iteration 16 ETA: [00:08:57]
Iteration 17 ETA: [00:09:03]
Iteration 18 ETA: [00:08:45]
Iteration 19 ETA: [00:08:34]
Iteration 20 ETA: [00:08:37]
Iteration 21 ETA: [00:08:22]
Iteration 22 ETA: [00:08:19]
Iteration 23 ETA: [00:08:18]
Iteration 24 ETA: [00:08:18]
Iteration 25 ETA: [00:08:24]
Iteration 26 ETA: [00:08:21]
Iteration 27 ETA: [00:08:53]
Iteration 28 ETA: [00:08:13]
Iteration 29 ETA: [00:08:19]
Iteration 30 ETA: [

In [20]:
order_0_test = torch.cat(order_0, dim=0)
orders_1_and_2_test = torch.cat(orders_1_and_2, dim=0)

order_0_test = order_0_test.cpu().numpy()
orders_1_and_2_test = orders_1_and_2_test.cpu().numpy()

In [21]:
order_0_test = order_0_test.reshape((n_molecules_test, -1))
orders_1_and_2_test = orders_1_and_2_test.reshape((n_molecules_test, -1))

In [22]:
basename = 'molecule_L_{}_J_{}_sigma_{}_MNO_{}_powers_{}.npy'.format(
        L, J, sigma, (M, N, O), integral_powers)

cache_dir = get_cache_dir("results/test/")

filename = os.path.join(cache_dir, 'order_0_' + basename)
np.save(filename, order_0_test)

filename = os.path.join(cache_dir, 'orders_1_and_2' + basename)
np.save(filename, orders_1_and_2_test)